In [ ]:
# what I got from Raven 

In [1]:
from ravenpackapi import RPApi
from plyer import notification
import datetime
api = RPApi(api_key='Wy3ca5h5XMwXgYgNnqt4jZ',  product="edge") 

In [3]:
import logging
import time

from ravenpackapi import ApiConnectionError, RPApi
from ravenpackapi.utils.retry_logic import incremental_backoff

from ravenpackapi import RPApi
from plyer import notification
import datetime

api = RPApi(api_key='Wy3ca5h5XMwXgYgNnqt4jZ',  product="edge") 

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# query the realtime feed

ds_EA = api.get_dataset(dataset_id="2ACA994D710FDC32890BB19BFFDA5B71")  # EnergyAccident
ds_RN = api.get_dataset(dataset_id="f24c292c0071493fadcc5d0c023ccd39")  # renew
ds_W  = api.get_dataset(dataset_id="5DF79AF7BFFB94967546F9F53E98CFF0")  # War_Con for oil producing countries 
# ds_EV

ds = api.get_dataset(dataset_id='us500-edge')    # "Climate Change"  "CleanTech"  # "us500" # "e22d52083bd14c7283996671b86fb102"

wait_time = incremental_backoff()
Rec_ID = []

while True:  
    try:
        for record in (ds.request_realtime() or ds_EA.request_realtime() or ds_W.request_realtime())  :
            if Rec_ID != record.TITLE:
                Rec_ID = record.TITLE
                rp_document_id = record["rp_document_id"] #record.RP_DOCUMENT_ID
                url = api.get_document_url(rp_document_id)
                '''
                @ Neeraj Send the notification from here
                '''
                print(record.timestamp_utc, record.entity_name, record.TITLE, record["event_relevance"], record.RP_DOCUMENT_ID)
                
    except (KeyboardInterrupt, SystemExit):
        break
    except ApiConnectionError as e:
        logger.error("Connection error %s: reconnecting..." % e)
        time.sleep(next(wait_time))

DEBUG:ravenpack.models.dataset:Connecting with RT feed: https://feed-edge.ravenpack.com/1.0/json/us500-edge?keep_alive
DEBUG:urllib3.connectionpool:Resetting dropped connection: feed-edge.ravenpack.com
DEBUG:urllib3.connectionpool:https://feed-edge.ravenpack.com:443 "GET /1.0/json/us500-edge?keep_alive HTTP/1.1" 200 None
DEBUG:ravenpack.core:Request get to /document/A9EA50DD47DAC97F051654C8E7AA82AC/url
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api-edge.ravenpack.com:443
DEBUG:urllib3.connectionpool:https://api-edge.ravenpack.com:443 "GET /1.0/document/A9EA50DD47DAC97F051654C8E7AA82AC/url HTTP/1.1" 404 None
INFO:ravenpack.core:API query to curl -X GET -H 'API_KEY:Wy3ca5h5XMwXgYgNnqt4jZ' 'https://api-edge.ravenpack.com/1.0/document/A9EA50DD47DAC97F051654C8E7AA82AC/url'
ERROR:ravenpack.core:Error calling the API, we tried: curl -X GET -H 'API_KEY:Wy3ca5h5XMwXgYgNnqt4jZ' 'https://api-edge.ravenpack.com/1.0/document/A9EA50DD47DAC97F051654C8E7AA82AC/url'


APIException404: Got an error 404: body was '{"endpoint":"document","errors":[{"reason":"Document not found","type":"ResourceNotFoundError"}]}
'